# Installation des bibliothèques Python
* Le point d'exclamation (!) signifie qu'on est en train d'écrire une instruction Shell au lieu d'un code Python. C'est comme si on est en train de faire une commande sur l'Invite des Commandes (cmd).

In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.5 MB/s eta 0:00:00


#Import des bibliothèques Python
* La bibliothèque BioPython (Bio) inclut plusieurs services permettant de naviguer les bases des données de National Center of Biotechnology information (NCBI).
* On va utiliser Entrez qui est une sous-bibliothèque permettant de traiter l'API Entrez de PubMed. On va également utiliser Medline qui est la base des données bibliographiques de PubMed.
* Avant d'utiliser l'Entrez API, veuillez définir votre adresse email en utilisant la variable d'exécution Entrez.email afin d'identifier vos requêtes de PubMed et que les administrateurs des ressources NCBI puissent vous contacter en cas d'abus de l'utilisation de l'API.

In [2]:
from Bio import Entrez
from Bio import Medline

Entrez.email = "your_email@example.com"

#Faire des recherches PubMed
* On utilise Entrez.esearch pour faire une requête du moteur de recherche PubMed.
  * On précise qu'on va utiliser PubMed comme la base des données en question.
  * On définit le terme à chercher. Il y a plusieurs astuces pour faire une très bonne formulation des termes de la recherche.
  * On identifie le nombre des résultats à afficher (retmax)
  * On choisit le critère du tri des résultats (sort).

In [3]:
def search_pubmed(query, num_results):
  handle = Entrez.esearch(db="pubmed", term=query, retmax=num_results, sort="relevance")
  record = Entrez.read(handle)
  handle.close()
  return record["IdList"]

#Extraire des données bibliographiques
* Pour extraire les données bibliographiques d'une publication indexée PubMed, on utilise la fonction Entrez.efetch. On précise:
  * L'identifiant PubMed de la Publication
  * PubMed comme la base des données
  * Medline comme le type de retour des données bibliographiques afin d'avoir toutes les données possibles
  * text (JSON) pour le format du résultat
* On utilise Medline.parse pour lire le retour Medline et le convertir en JSON.

In [4]:
def get_pubmed_metadata(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid, rettype="medline", retmode="text")
    records = Medline.parse(handle)
    record = next(records)
    handle.close()
    return record

#Extraire un champ bibliographique
* Pour retrouver l'identifiant du champ, veuillez vous référer à https://pubmed.ncbi.nlm.nih.gov/help/. Un exemple d'identifiant d'un champ est "TI" pour le titre et "MH" pour les mots-clés MeSH.
* Parfois, certains champs sont inaccessibles dans le Medline d'une publication indexée PubMed. On utilise "try ... except ..." pour se débarrasser de cette exception.
* Si vous voulez faire une boucle pour extraire un même champ bibliographique de plusieurs publications PubMed, n'oubliez pas qu'il y a une limite de requête. Il faut importer time et ajouter time.sleep(5) après chaque requête pour ajouter un temps de latence entre chaque requête.

In [6]:
query = "Artificial Intelligence"
num_results = 3
results = search_pubmed(query, num_results)

if results:
    first_result_pmid = results[0]
    metadata = get_pubmed_metadata(first_result_pmid)
try:
  print(metadata["TI"])
except KeyError:
  print("No title available")

Artificial intelligence to deep learning: machine intelligence approach for drug discovery.
